## 1.0 Collecting our data

Requesting data from the Austin, Texas API using requests.get

Both datasets contain around 160,000 rows of data. To retrieve all the data, using ```?$limit=160000``` at the end of both API endpoints

### 1.1 Requesting data from the intake API and storing as a JSON

In [ ]:
import requests
import pandas as pd

response = requests.get('https://data.austintexas.gov/resource/wter-evkm.json?$limit=160000')
data_set1 = response.json()
data_set1

### 1.2 Requesting data from the outcome API and storing as a JSON

In [ ]:
response2 = requests.get('https://data.austintexas.gov/resource/9t4d-g238.json?$limit=160000')
data_set2 = response2.json()
data_set2

### 1.3 Converting JSON data to a dataframe using PANDAS

In [ ]:
# dataframe(df) for intake data
animal_intake_df = pd.DataFrame(data_set1)

# only showing the first 5 rows - can remove .head() to see more rows
animal_intake_df.head(5)


In [ ]:
# dataframe(df) for outcomes data

animal_outcome_df = pd.DataFrame(data_set2)

animal_outcome_df.head(5)


### 1.4 Writing datasets to csv files

In [ ]:
# write intake data to csv
animal_intake_df.to_csv("animal_intake_dataset.csv")

# write outcome data to csv
animal_outcome_df.to_csv("animal_outcome_dataset.csv")


## 2.0 Cleaning the intake dataset

In [ ]:
# checking the datatypes of the columns
animal_intake_df.info()


### 2.1 Identifying rows with duplicate values in the animal_id and datetime columns in the intakes:

In [ ]:
# Identify duplicate rows based on the 'animal_id' and 'datetime' columns:
duplicates_intakes = animal_intake_df[animal_intake_df.duplicated(subset=['animal_id', 'datetime', 'datetime2'], keep=False)]

duplicates_intakes_sorted = duplicates_intakes.sort_values(by=['animal_id', 'datetime'])

In [ ]:
# Converting the result into a dataframe and showing the result to observe the duplicates
duplicate_intakes_df = pd.DataFrame(duplicates_intakes_sorted)

duplicate_intakes_df.head(20)

### 2.2 Removing duplicate rows from the intakes df

Removing duplicate rows where the animal_id and datetime values are the same.
Where they are the same, this means the data was inputted twice (probably by mistake) and so removing these duplicate values will ensure the results are not skewed.

In [ ]:
# show key features-based duplicates with identical animal_id, datetime, datetime2 (except for first occurrence)
animal_intake_df[animal_intake_df[["animal_id", "datetime", "datetime2"]].duplicated()]

In [ ]:
# delete duplicates and save changes to existing dataframe
animal_intake_df.drop_duplicates(subset=['animal_id', 'datetime', 'datetime2'], inplace=True)
animal_intake_df

### 2.3 Removing duplicate columns from the intakes df

The dataset contains two seemingly identical columns for datetime of intake: datetime and datetime2. Confirm that these are in fact duplicates, then delete duplicate columns and replace with a more appropriately named intake_date column.

In [ ]:
# checking to ensure datetime and datetime2 are identical for all records
animal_intake_df[~(animal_intake_df["datetime"] == animal_intake_df["datetime2"])]

In [ ]:
# storing the datetime column data into a variable intake_date
intake_date = animal_intake_df['datetime']

# using the PANDAS drop function to drop the 'datetime' and 'datetime2' columns from the intake dataset
intake_df = animal_intake_df.drop(['datetime', 'datetime2'], axis='columns')

# inserting the new column which we created above (intake_date) and placing this at index 1 in the dataframe
intake_df.insert(1, 'intake_date', intake_date)

intake_df.shape


## 3.0 Cleaning the outcome dataset

Repeat the process that was completed above in the intakes df for the outcomes dataset

### 3.1 Removing duplicate rows from the outcome df

Removing duplicate rows where the animal_id and datetime values are the same.
Where they are the same, this means the data was inputted twice (probably by mistake) and so removing these duplicate values will ensure the results are not skewed.

In [ ]:
# show key features-based duplicates with identical animal_id, datetime, monthyear (except for first occurrence)
animal_outcome_df[animal_outcome_df[["animal_id", "datetime", "monthyear"]].duplicated()]

In [ ]:
# delete duplicates and save changes to existing dataframe
animal_outcome_df.drop_duplicates(subset=['animal_id', 'datetime', 'monthyear'], inplace=True)
animal_outcome_df

### 3.2 Removing duplicate columns from the outcomes df

The dataset contains two seemingly identical columns for datetime of intake: datetime and monthyear. Confirm that these are in fact duplicates, then delete duplicate columns and replace with a more appropriately named intake_date column.

In [ ]:
# checking to ensure datetime and monthyear are identical for all records
animal_outcome_df[~(animal_outcome_df["datetime"] == animal_outcome_df["monthyear"])]

In [ ]:
# storing the datetime column data into a variable outcome date
outcome_date = animal_outcome_df['datetime']

# using the PANDAS drop function to drop the 'datetime' and 'monthyear' columns from the outcome dataset
outcome_df = animal_outcome_df.drop(['datetime', 'monthyear'], axis='columns')

# inserting the new column which we created above (outcome_date) and placing this at index 1 in the dataframe
outcome_df.insert(1, 'outcome_date', outcome_date)

outcome_df.shape

## 4.0 Merging intake and outcome dataframes

For ease of analysis, the outcome and intake dataframes are merged. We choose to perform an inner join on animal_id to limit the data that we work with to those animals where there is information about both their intake and their ultimate outcome.

In [ ]:
# merging dataframes based on animal_id
shelter_df = pd.merge(intake_df, outcome_df, how='inner', on='animal_id', suffixes=['_outcomes', '_intakes'])

shelter_df

## 5.0 Cleaning merged dataframe

### 5.1 Converting animal_id to data type integer

Remove the "A" at the beginning of the animal_id by using regex. We can then turn the animal_id into an integer data type for ease of sorting and manipulation.

In [ ]:
# Use of regex to remove the A in animal_id, rather than using string slicing as if the code is run multiple times, then will remove a value each time
shelter_df['animal_id'] = shelter_df['animal_id'].str.replace(r'A', '', regex=True).astype(int)

shelter_df

### 5.2 Removing duplicate and unneeded columns

The two datasets had some columns that we thought might contain the exact same data, thereby introducing duplicates upon the merge. We checked these columns to confirm if the data was in fact duplicate, then deleted duplicates accordingly.

In [ ]:
# Confirm that color_outcomes and color_intakes are duplicates (result confirms duplication)
shelter_df[~(shelter_df["color_outcomes"] == shelter_df["color_intakes"])]

In [ ]:
# Confirm that breed_outcomes and breed_intakes are duplicates (result confirms duplication)
shelter_df[~(shelter_df["breed_outcomes"] == shelter_df["breed_intakes"])]

In [ ]:
# Confirm that animal_type_outcomes and animal_type_intakes are duplicates (result confirms duplication)
shelter_df[~(shelter_df["animal_type_outcomes"] == shelter_df["animal_type_intakes"])]

In [ ]:
# Cheking if sex_upon_outcome and sex_upon_intake are duplicates (result confirms NOT duplicated)
shelter_df[~(shelter_df["sex_upon_outcome"] == shelter_df["sex_upon_intake"])]

In [ ]:
# Removing duplicate columns
shelter_df.drop(['color_outcomes',
                'breed_intakes',
                'animal_type_outcomes'],
                axis='columns',
                inplace=True)

shelter_df


Separately, there are some columns in the merged dataframe that are not helpful for our analysis. These include name_intakes, name_outcomes, and found_location. We remove these columns.

In [ ]:
# Removing columns that we do not need for our analysis
shelter_df.drop(['name_intakes',
                'name_outcomes',
                'found_location'],
                axis='columns',
                inplace=True)

shelter_df

### 5.3 Removing unnecessary records
We remove unnecessary records from our dataset that are not directly relevant to our primary goal of increasing adoption rates. As we know there are some animals that come under the category of livestock and wildlife, we will check how many of the animals were adopted and if there is an insufficient amount of data, the rows will be removed. 

In [ ]:
# Check adoptions by intake_type
shelter_df.groupby(["intake_type", "outcome_type"])["outcome_type"].count()

In [ ]:
# Examine wildlife that was adopted
shelter_df[(shelter_df["intake_type"] == "Wildlife") & (shelter_df["outcome_type"] == "Adoption")]

In [ ]:
# Check adoptions by animal type
shelter_df.groupby(["animal_type_intakes", "outcome_type"])["outcome_type"].count()

In [ ]:
# Examine livestock that was adopted
shelter_df[(shelter_df["animal_type_intakes"] == "Livestock") & (shelter_df["outcome_type"] == "Adoption")]

As expected, there are a a few amount of wildlife and livestock animals that were adopted and due to this, we will remove these rows from our dataset. 

In [ ]:
# Deleting rows that contain Livestock animal data
shelter_df.drop(shelter_df[shelter_df["animal_type_intakes"] == "Livestock"].index, inplace=True)

shelter_df[shelter_df["animal_type_intakes"] == "Livestock"]

In [ ]:
# Deleting rows that contain Wildlife animal data
shelter_df.drop(shelter_df[shelter_df["animal_type_intakes"] == "Wildlife"].index, inplace=True)

shelter_df[shelter_df["animal_type_intakes"] == "Wildlife"]

When we looked at the aoutcome type for the animals, there was an outcome type 'RTO-adopt'. After doing some research there is no clear answer as to what this may mean.
Therefore, we removed the rows of data where to outcome type included RTO-Adopt to avoid confusion when wanting to look at adoption rates during the analysis as it is not clear if these animals were adopted or returned to their owner. 

In [ ]:
shelter_df.drop(shelter_df[shelter_df["outcome_type"] == "Rto-Adopt"].index, inplace=True)

shelter_df[shelter_df["outcome_type"] == "Rto-Adopt"]

### 5.4 Removing whitespaces

The data in our datasets are likely to be input by users. To ensure that leading and trailing whitespaces do not interfere with our analysis, we define a lambda function to remove them using the .apply method.

In [ ]:
# look for columns with datatype object and strip any leading/trailing spaces
shelter_df.apply(lambda value: value.str.strip() if value.dtype == "object" else value)

shelter_df.head(3)

## 6.0 Additional data manipulation to prepare for analysis

We identified several ways that the data needed to be manipulated to prepare it for our analysis.

### 6.1 Extracting sex and neuter status into separate columns
One of our observations was that the sex_upon_intake and sex_upon_outcome contain the sex of the animal as well as their neuter status. What is important for our analysis is sex of each animal as well as neuter status at the time of outcome. We prepare our data by (i) seperating sex into separate columns for both intake and outcome, (ii) comparing the values to confirm they are the same, then (iii) deleting one of the duplicate columns. For neuter status, we create a new column for neuter status at outcome.

In [ ]:
# using regex to return values about neuturing into new columns -
# this is done by extracting the word before the white space as can see from the data that there is a space between the neuteruing and gender of animal
shelter_df['Neuter Status at Outcome'] = shelter_df['sex_upon_outcome'].str.extract('(\w*\s)', expand=True)

# using regex to return the sex of the animal at intake and outcome in new columns
shelter_df['Sex at Intake'] = shelter_df['sex_upon_intake'].str.extract(('[^ ]* (.*)'), expand=True)
shelter_df['Sex at Outcome'] = shelter_df['sex_upon_outcome'].str.extract(('[^ ]* (.*)'), expand=True)

shelter_df

Some values in these new columns are missing, so we replace NaN values with "Unknown"

In [ ]:
# adding the string Unknown into as the value for new columns that contain NaN
shelter_df[['Sex at Intake', 'Sex at Outcome', 'Neuter Status at Outcome']] = shelter_df[['Sex at Intake', 'Sex at Outcome', 'Neuter Status at Outcome']].fillna('Unknown')

shelter_df

In [ ]:
# Confirm that sex at intake and sex at outcome are the same for each record (result confirms these columns are duplicates)
shelter_df[~(shelter_df["Sex at Intake"] == shelter_df["Sex at Outcome"])]

In [ ]:
# Delete duplicate column as well as source columns which are no longer needed
shelter_df.drop(['Sex at Outcome',
                'sex_upon_outcome',
                'sex_upon_intake'],
                axis='columns',
                inplace=True)

shelter_df

### 6.2 Calculating each animal's length of stay at the shelter

We can find the amount of time each animal spent in the shelter and store this in a new column. For this calculation, we convert the intake_date and outcome_date columns to datetime using the `to_datetime` function, then subtract the outcome date by the intake date.

In [ ]:
shelter_df['intake_date'] = pd.to_datetime(shelter_df['intake_date'])
shelter_df['outcome_date'] = pd.to_datetime(shelter_df['outcome_date'])
shelter_df['Time In Shelter'] = shelter_df['outcome_date'] - shelter_df['intake_date']
shelter_df

We can now get a sense of the distribution in our data by using the describe function.


In [ ]:
shelter_df.describe()

This shows the minimum time in shelter is negative. Where the time in shelter is negative, the intake date is more recent then the outcome date. We check for records where the intake date is after the outcome date.

In [ ]:
# check for occurrences where intake date is after outcome date (suggesting data input error)
incorrect_dates = shelter_df[shelter_df['intake_date'] >= shelter_df['outcome_date']]

incorrect_dates

There are a considerable number of rows with this discrepancy. Because these inaccuracies could skew our analysis, we choose to remove these records.

In [ ]:
# Using .index, we remove the rows by the index of the incorrect date rows
shelter_df.drop(incorrect_dates.index, inplace=True)
shelter_df

In [ ]:
# check the distribution of data again using describe function (result shows no remaining negative values)
shelter_df.describe()

### 6.3 Sense-checking animals' birth dates and intake/outcome dates

Because of the date inconsistencies that we found when we compared intake and outcome dates, we decided to sense-check the intake/outcome dates against animals' dates of birth. Where there are birth dates that come after outcome dates or intake dates, these are likely to be errors. We therefore delete records where this is the case.

In [ ]:
# identify instances where date of birth comes after outcome date
birth_outcome_intake_date_discrepancies = shelter_df[(shelter_df['date_of_birth'] > shelter_df['outcome_date']) | (shelter_df['date_of_birth'] > shelter_df['intake_date'])]

birth_outcome_intake_date_discrepancies

In [ ]:
# Using .index to remove the rows by the index of the rows with the inconsistent dates
shelter_df.drop(birth_outcome_intake_date_discrepancies.index, inplace=True)

shelter_df

### 6.4 Convert age at intake and age at outcome to usable formats

The age data in the raw datasets is not ideal for analysis because there is no standard unit (some ages are in weeks, others in months or years). We perform manipulations to get the ages in a common unit of measure - days. Then, we use our new columns to assign each animal to an age group.

#### Convert ages to units in days

In [ ]:
# convert date of birth to datetime
shelter_df['date_of_birth'] = pd.to_datetime(shelter_df['date_of_birth'])

# calculate age at intake and age at outcome using birth date
shelter_df['Age at Intake'] = shelter_df['intake_date'] - shelter_df['date_of_birth']
shelter_df['Age at Outcome'] = shelter_df['outcome_date'] - shelter_df['date_of_birth']

shelter_df

In [ ]:
# check statistics for newly calculated ages at intake and outcome
shelter_df.describe()

#### Create age ranges for animals at intake and outcome

In [ ]:
# define a bins list for the age ranges we want in years
bins_yrs = [0, 0.25, 0.5, 1, 2, 3, 5, 7, 10, 13, 16, 19, 31]

# convert the bins to days since that is the unit of measure of our data
bins_days = [round(yrs * 365.25) for yrs in bins_yrs]

bins_days

In [ ]:
# define string labels for the bins
labels = ["0-3 mon", "3-6 mon", "6-12 mon", "1-2 yrs", 
          "2-3 yrs", "3-5 yrs", "5-7 yrs", "7-10 yrs", 
          "10-13 yrs", "13-16 yrs", "16-19 yrs", "19-31 yrs"
         ]

In [ ]:
# add new columns for age ranges at intake and at outcome; and assign age ranges using cut
shelter_df["Age Range at Intake"] = pd.cut(shelter_df["Age at Intake"], bins=pd.to_timedelta(bins_days, unit="D"), labels=labels, right=False)
shelter_df["Age Range at Outcome"] = pd.cut(shelter_df["Age at Outcome"], bins=pd.to_timedelta(bins_days, unit="D"), labels=labels, right=False)

# show a sample of records for relevant columns as a spot check
shelter_df[["Age at Intake", "Age Range at Intake", "Age at Outcome", "Age Range at Outcome"]].sample(15)


In [ ]:
# delete age_upon_intake, age_upon_outcome, and date_of_birth which are now redundant
shelter_df.drop(['age_upon_intake',
                'age_upon_outcome',
                'date_of_birth'],
                axis='columns',
                inplace=True)

shelter_df

### 6.5 Identifying purebred vs mixed breed animals
A characteristic that may be useful for our analysis is to distingish animals from those who are mixed breed versus purebred. We use the breed information from within our dataframe to create a new column with boolean values where True signifies mixed breed and False signifies purebred.

In [ ]:
# create new column that distinguishes mixed breed from purebred
shelter_df['Mixed Breed'] = shelter_df['breed_outcomes'].str.contains('Mix', case=True, regex=False)
shelter_df[['breed_outcomes', 'Mixed Breed']].sample(10)

## 7.0 Addressing null values

### 7.1 Identifying null values

We identify and explore records with null values so that we can make informed decisions about how to address the missing data.

In [ ]:
# generate a heatmap of missing values using matplotlib and seaborn
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(30,30))
sns.heatmap(shelter_df.isnull(), cbar=False, cmap="YlGnBu")
plt.show()

The heatmap suggests that missing values are concentrated in the outcome_subtype column, but there are also some missing values in the outcome_type column. We look more closely at the numbers of missing values in the two columns using the .isna() and .sum() method.

In [ ]:
# check for non-null values
null_val_shelter = shelter_df.isnull().sum()
null_val_shelter

In [ ]:
# checking how many rows have null values in outcome_type columns:
rows_with_nulls = shelter_df[shelter_df['outcome_type'].isnull()].any(axis=1)
null_rows_count = rows_with_nulls.sum()
null_rows_count

### 7.2 Addressing null outcome_type values

outcome_type is a very important column for our analysis, and for this reason, we choose to delete the 36 rows that have null values in this column.

In [ ]:
# delete the rows that have missing values for outcome_type
shelter_df.dropna(subset=['outcome_type'], inplace=True)

### 7.3 Addressing null outcome_subtype values

We explore the outcome_subtype data to determine its relevance and usefulness to our analysis. First we replace all null values with "Not specified", then we aggregate the data to see what we can learn.

In [ ]:
# replace null values for outcome_subtype with "Not specified"
shelter_df["outcome_subtype"] = shelter_df["outcome_subtype"].fillna("Not specified")

In [ ]:
# aggregate the data
shelter_df.groupby(["outcome_type", "outcome_subtype"])["outcome_subtype"].count()

We generated a bar chart to illustrate the outcome subtype for those animals who were adopted. 

In [ ]:
# Filter the df to select only 'Adoption' outcomes:
adoption_df = shelter_df[shelter_df['outcome_type'] == 'Adoption']

# Count the occurrences of each outcome_subtype for Euthanasia cases
outcome_subtype_counts = adoption_df['outcome_subtype'].value_counts()

# Choose the top N most common outcome_subtype values to display
top_n = 10
top_n_values = outcome_subtype_counts.head(top_n)

# Create a bar chart
plt.bar(top_n_values.index, top_n_values.values)
plt.xlabel('Outcome Subtype')
plt.ylabel('Count')
plt.title('Top ' + str(top_n) + ' Outcome Subtype Distribution for Adoption')
plt.xticks(rotation=45)
plt.tight_layout()

# Display the plot
plt.show()

Because the majority of adoption subtypes are "Not specified" we decided that this information is not helpful to our analysis. For this reason, we chose to delete outcome_subtype from our dataset.

In [ ]:
# delete outcome_subtype which is not useful for our analysis
shelter_df.drop(['outcome_subtype'],
                axis='columns',
                inplace=True)

shelter_df

## 8.0 Some final cleaning steps for readability and ease of use

### 8.1 Changing column names

In [ ]:
# lets quickly check what column names our dataframe currently has:
column_names = shelter_df.columns

for column_name in column_names:
    print(column_name)

To improve readability of the data, we rename the columns.

In [ ]:
# change column names to improve readability
shelter_df.rename(columns={'animal_id':'Animal ID',
                           'intake_date': 'Intake Date',
                           'found_location': 'Found Location',
                           'intake_type': 'Intake Type',
                           'intake_condition': 'Intake Condition',
                           'breed_outcomes':'Breed',
                           'outcome_date': 'Outcome Date',
                           'outcome_type': 'Outcome Type',
                           'animal_type_intakes': 'Animal',
                           'color_intakes': 'Colour',
                           'Sex at Intake': 'Sex'},
                  inplace=True)

shelter_df.head(3)

### 8.2 Sorting rows

We sort the rows based on the intake date and reset the index as the old index is not needed.

In [ ]:
# sort by Intake Date
shelter_df.sort_values('Intake Date', inplace=True)

# reset index
shelter_df.reset_index(drop=True, inplace=True)

shelter_df.head(5)

#### 8.2.1 Addressing an observation: potential duplicates in data

When looking at the data above, we noticed rows 1-3 are identical except for animal id. We then used the below code to check if there were any more rows in the data like this and saw that there were around 3000 potential duplicates.

When looking further into this, we noticed that the majority had a very young Intake Age. We thought that this is likely to mean that the animals were brought to the shelter as a litter. Therefore we chose not remove these rows from our data.

In [ ]:
shelter_df[shelter_df.duplicated(subset=shelter_df.columns.difference(['Animal ID']))]

### 8.3 Rearranging column postions

We rearrange the column positions for easier readability of the data.

In [ ]:
shelter_df = shelter_df.reindex(columns=['Animal ID',
                                         'Animal',
                                         'Intake Date',
                                         'Outcome Date',
                                         'Time In Shelter',
                                         'Age at Intake',
                                         'Age Range at Intake',
                                         'Age at Outcome',
                                         'Age Range at Outcome',
                                         'Intake Condition',
                                         'Intake Type',
                                         'Outcome Type',
                                         'Sex',
                                         'Neuter Status at Outcome',
                                         'Breed',
                                         'Mixed Breed',
                                         'Colour'])

shelter_df

## 9.0 Write cleaned data to csv file

In [ ]:
# write cleaned data to csv
shelter_df.to_csv('Austin_Animal_Center_clean_data.csv')